In [231]:
import numpy as np
import pandas as pd
import mylib as ml

train = pd.read_csv('DATA/Entrenamieto_ECI_2020.csv')
test = pd.read_csv('DATA/Validacion_ECI_2020.csv')
o_train = train.copy()
o_test = test.copy()

pd.options.display.float_format = '{:,.3f}'.format

In [232]:
class Analizador:

    def __init__(self, train, test):
        self.train = train
        self.test = test

    def get_cols_date(self):
        return [x for x in train.columns if "Date" in x]

    def print_duplicates_count(self):
        print("Cantidad de duplicados")
        print(len(self.train)-len(self.train.drop_duplicates()))
        print(len(self.test)-len(self.test.drop_duplicates()))
        print("No hay duplicados")

    def eliminar_columnas_que_no_cambian(self):
        print('Eliminia las columnas de [test y train] que no aportan informacion en train.')
        non_variant_cols = self.train.columns[train.nunique() <= 1]
        print(non_variant_cols)
        train.drop(non_variant_cols, axis=1, inplace=True)
        test.drop(non_variant_cols, axis=1, inplace=True)

    def eliminar_all_outliers(self, n=4):
        for c in self.train.select_dtypes('number').columns:
            self.eliminar_outliers(c, n)

    def eliminar_outliers(self, col, n=4):
        self.train = self.train[np.abs(self.train[col]-self.train[col].mean()) < n*self.train[col].std()]

    def eliminar_outliers_p(self, df, col, p=0.001):
        l,h = p if type(p) is list else [p,1-p]
        q_low = df[col].quantile(l)
        q_hi  = df[col].quantile(h)
        return df[(df[col] < q_low) | (df[col] > q_hi) ]
    
    def convertir_fechas(self, cols_date):
        for df in [self.train, self.test]:
            for date_col in cols_date:
                df[date_col] = pd.to_datetime(df[date_col])

    def ampliar_info_fechas(self, cols_date):
        for df in [self.train, self.test]:
            for x in cols_date:
                df[x+'_dow'] = df[x].dt.dayofweek 
                df[x+'_moy'] = df[x].dt.month 


    def agregar_datos_de_cuentas(self):
        part_win = self.train[self.train.Stage=='Closed Won'].groupby(['Account_Name']).Stage.count()
        part = self.train.groupby(['Account_Name']).Stage.count()
        accounts = pd.concat([part,part_win],axis=1)
        accounts.columns = ['participo','gano']
        accounts.fillna(0,inplace=True)
        accounts['gano_p'] = accounts.gano/accounts.participo
        accounts
        for c in accounts.index:
            for df in [self.train,self.test]:
                df.loc[df.Account_Name==c,'participo'] = accounts.loc[c,'participo']
                df.loc[df.Account_Name==c,'gano'] = accounts.loc[c,'gano']
                df.loc[df.Account_Name==c,'gano_p'] = accounts.loc[c,'gano_p']

    def agregar_datos_de_fechas(self):
        for df in [self.train,self.test]:
            df['f_days_from_client_created'] = (df['Opportunity_Created_Date'] - df['Account_Created_Date']).dt.days
            # dias desde la ultima actividad
            df['f_days_from_last_activity'] = (df['Last_Modified_Date'] - df['Opportunity_Created_Date']).dt.days
            # dias que dura el presupuesto
            df['f_days_expiry_duration'] = (df['Quote_Expiry_Date'] - df['Opportunity_Created_Date']).dt.days 


            df['planned_days'] = (df['Planned_Delivery_End_Date'] - df['Planned_Delivery_Start_Date']).dt.days
            #agrupamientos
            tg = df.groupby('Opportunity_ID')
            df['Total_Amount_sum'] = tg.Total_Amount.transform('sum')

            df['planned_days_mean'] = tg.planned_days.transform('mean')
            df['planned_days_median'] = tg.planned_days.transform('median')
            df['planned_days_max'] = tg.planned_days.transform('max')
            df['planned_days_min'] = tg.planned_days.transform('min')

            df['min_planned_start_Date'] = tg.Planned_Delivery_Start_Date.transform('min')
            df['max_planned_end_Date'] = tg.Planned_Delivery_End_Date.transform('max')
            df['planned_g_diff'] = (df['max_planned_end_Date'] - df['min_planned_start_Date']).dt.days
        
        
Ana = Analizador(train,test)


Analizo de que manera agrupa "Opportunity_ID", que es el atributo que tengo que estimar

In [233]:
print('cuales cambian en train')
print(train.columns[(train.groupby('Opportunity_ID').nunique() > 1).any()])
print('cuales cambian en test')
print(test.columns[(test.groupby('Opportunity_ID').nunique() > 1).any()])

print("Valores unicos de Opportunity_ID")
print(train.groupby('Opportunity_ID').count().shape[0])

cuales cambian en train
Index(['ID', 'Product_Family', 'Product_Name', 'ASP', 'ASP_(converted)',
       'Planned_Delivery_Start_Date', 'Planned_Delivery_End_Date', 'Month',
       'Delivery_Quarter', 'Delivery_Year', 'TRF', 'Total_Amount'],
      dtype='object')
cuales cambian en test
Index(['ID', 'Product_Family', 'Product_Name', 'ASP', 'ASP_(converted)',
       'Planned_Delivery_Start_Date', 'Planned_Delivery_End_Date', 'Month',
       'Delivery_Quarter', 'Delivery_Year', 'TRF', 'Total_Amount'],
      dtype='object')
Valores unicos de Opportunity_ID
9841


Cambian las siguientes:

* Product_Family
* Product_Name
* ASP
* ASP_(converted)
* Planned_Delivery_Start_Date
* Planned_Delivery_End_Date
* Month
* Delivery_Quarter
* Delivery_Year
* TRF
* Total_Amount

Analizo estas columnas


In [234]:
Ana.eliminar_columnas_que_no_cambian()

cols_date = [x for x in train.columns if x.endswith('_Date')]
Ana.convertir_fechas(cols_date)
Ana.ampliar_info_fechas(cols_date)



Eliminia las columnas de [test y train] que no aportan informacion en train.
Index(['Submitted_for_Approval', 'Last_Activity', 'ASP_(converted)_Currency',
       'Actual_Delivery_Date', 'Prod_Category_A'],
      dtype='object')


In [236]:
Ana.train.columns

Index(['ID', 'Region', 'Territory', 'Pricing, Delivery_Terms_Quote_Appr',
       'Pricing, Delivery_Terms_Approved', 'Bureaucratic_Code_0_Approval',
       'Bureaucratic_Code_0_Approved', 'Bureaucratic_Code',
       'Account_Created_Date', 'Source ', 'Billing_Country', 'Account_Name',
       'Opportunity_Name', 'Opportunity_ID', 'Sales_Contract_No',
       'Account_Owner', 'Opportunity_Owner', 'Account_Type',
       'Opportunity_Type', 'Quote_Type', 'Delivery_Terms',
       'Opportunity_Created_Date', 'Brand', 'Product_Type', 'Size',
       'Product_Category_B', 'Price', 'Currency', 'Quote_Expiry_Date',
       'Last_Modified_Date', 'Last_Modified_By', 'Product_Family',
       'Product_Name', 'ASP_Currency', 'ASP', 'ASP_(converted)',
       'Planned_Delivery_Start_Date', 'Planned_Delivery_End_Date', 'Month',
       'Delivery_Quarter', 'Delivery_Year', 'TRF', 'Total_Amount_Currency',
       'Total_Amount', 'Total_Taxable_Amount_Currency', 'Total_Taxable_Amount',
       'Stage', 'Account_

In [237]:
print('Revisar Año max 2208 en Planned_Delivery_End_Date en train')
train[train['Planned_Delivery_End_Date'] > pd.to_datetime('2022-01-01') ]['Planned_Delivery_End_Date']

print('Corrijo fecha')
train.at[15370,'Planned_Delivery_End_Date'] = pd.to_datetime('2018-12-31')


Revisar Año max 2208 en Planned_Delivery_End_Date en train
Corrijo fecha


In [238]:
for df in [train,test]:
    df.Sales_Contract_No = pd.to_numeric(df.Sales_Contract_No, errors='coerce')
    df['con_contrato'] = df.Sales_Contract_No > 0

In [239]:
print(train[['con_contrato','Stage']].groupby('Stage').sum())
print('El atributo "Sales_Contract_nro", evidentemente esta ligado al Closed Won. Probablemente, se asigne este numero, cuando se concreta la venta. Por lo tanto no se usa como feature ')
for df in [train,test]:
    df.drop(['con_contrato','Sales_Contract_No'], axis=1, inplace=True)

con_contrato
Stage                      
Closed Lost         579.000
Closed Won        9,394.000
Negotiation           0.000
Proposal              0.000
Qualification         0.000
El atributo "Sales_Contract_nro", evidentemente esta ligado al Closed Won. Probablemente, se asigne este numero, cuando se concreta la venta. Por lo tanto no se usa como feature 


In [240]:
#elimino las fechas
for df in [train,test]:
    df.drop(cols_date, axis=1, inplace=True)

In [241]:
print(train.shape)
a = Ana.eliminar_all_outliers(1)
print(train.shape)
#An.eliminar_all_outliers(4)
NO ELIMINA NADA ????


SyntaxError: invalid syntax (<ipython-input-241-b5175bb96445>, line 5)

In [242]:
print('revisar estos casos')
tax_mayor_a_total = train[(np.abs(train.Total_Taxable_div) > 2) & train.Total_Taxable_Amount > 0]
tax_mayor_a_total[['Opportunity_ID','Total_Taxable_Amount','Total_Amount_sum','Total_Taxable_div','Stage']]



revisar estos casos


AttributeError: 'DataFrame' object has no attribute 'Total_Taxable_div'

In [243]:
col_fijas_train = train.columns[(train.groupby('Opportunity_ID').nunique() <= 1).all()]
col_fijas_test = test.columns[(test.groupby('Opportunity_ID').nunique() <= 1).all()]
df_train = train[col_fijas_train].drop_duplicates().set_index('Opportunity_ID')
df_test = test[col_fijas_test].drop_duplicates().set_index('Opportunity_ID')

#df_train = train.set_index('Opportunity_ID')
#df_test = test.set_index('Opportunity_ID')

#Elimino identificadoes
#cols_to_drop =  ['ID', 'Opportunity_ID', 'Opportunity_Name']
#df_train.drop(cols_to_drop, axis=1, inplace=True)
#df_test.drop(cols_to_drop, axis=1, inplace=True)
#col_fijas_train

In [244]:
#train.to_csv('output/train_minable.csv', index=False)
#test.to_csv('output/test_minable.csv', index=False)
#df_train.to_csv('output/df_train_minable.csv')
#df_test.to_csv('output/df_test_minable.csv')

#df = pd.concat([df_test,df_train], axis=0, ignore_index=True)
#df.loc[df.Stage.isna(),'Stage'] = 'FALTA'


In [245]:
#solo dejo los Stage Win y Lost
print(df_train.Stage.value_counts())
df_train = df_train[df_train.Stage.isin(['Closed Won','Closed Lost'])]
print(df_train.Stage.value_counts())

Closed Won       5072
Closed Lost      4719
Proposal           30
Negotiation        11
Qualification       9
Name: Stage, dtype: int64
Closed Won     5072
Closed Lost    4719
Name: Stage, dtype: int64


In [246]:
An.agregar_datos_de_cuentas(train, test)

In [247]:
X = df_train.drop('Stage', axis=1)
#y = df_train['Stage'].apply(lambda x: 1 if x == 'Closed Won' else 0)
y = df_train['Stage']
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3)
X_test = df_test

In [248]:

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

In [249]:

#features_to_ignore=['con_contrato','Sales_Contract_No']
#for df in [X_train, X_valid, X_test]:
#    df.drop(features_to_ignore, axis=1, inplace=True)


numeric_features = X_train.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X_train.select_dtypes(include=['object','bool']).columns


In [250]:
from sklearn.compose import ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
        ])

In [251]:

from sklearn.ensemble import RandomForestClassifier
#cls_rf = RandomForestClassifier(n_estimators=150, criterion='gini')
cls_rf = RandomForestClassifier()
rf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', cls_rf)])

rf.fit(X_train, y_train)
y_pred = rf.predict(X_valid)
rf.score(X_valid, y_valid)


0.8488767869298843

In [228]:
numeric_features
#categorical_features

Index(['Pricing, Delivery_Terms_Quote_Appr',
       'Pricing, Delivery_Terms_Approved', 'Bureaucratic_Code_0_Approval',
       'Bureaucratic_Code_0_Approved', 'Total_Taxable_Amount',
       'Account_Created_Date_dow', 'Account_Created_Date_moy',
       'Opportunity_Created_Date_dow', 'Opportunity_Created_Date_moy',
       'Quote_Expiry_Date_dow', 'Quote_Expiry_Date_moy',
       'Last_Modified_Date_dow', 'Last_Modified_Date_moy', 'participo', 'gano',
       'gano_p'],
      dtype='object')

In [177]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree

cls_dt = DecisionTreeClassifier()
dt = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', cls_dt)
    ])

dt.fit(X_train, y_train)

y_pred = dt.predict(X_valid)
print(dt.score(X_valid, y_valid))

#tree.export_text(cls_dt,max_depth=0)

0.8787759131293189


In [172]:


from sklearn.metrics import accuracy_score, log_loss
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

pipes=[]
classifiers = [
    #KNeighborsClassifier(3),
    SVC(kernel="rbf", C=0.025, probability=True),
    #NuSVC(probability=True),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    AdaBoostClassifier(),
    GradientBoostingClassifier()
    ]
for classifier in classifiers:
    pipe = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', classifier)])
    pipe.fit(X_train, y_train)  
    pipes.append(pipe) 
    print(classifier)
    print("model score: %.3f" % pipe.score(X_valid, y_valid))
    print("-"*60)


SVC(C=0.025, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)
model score: 0.791
------------------------------------------------------------
DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')
model score: 0.879
------------------------------------------------------------
RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
           

In [174]:
cls_model=pipes[2]

param_grid = { 
    'classifier__n_estimators': range(10,200,20),
    #'classifier__max_depth': range(10,170,10),
    #'classifier__max_features': ['auto', 'sqrt', 'log2'],
    #'classifier__max_depth' : [4,5,6,7,8],
    'classifier__criterion' :['gini', 'entropy']
    }

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from time import time

# Utility function to report best scores
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})"
                  .format(results['mean_test_score'][candidate],
                          results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")


# specify parameters and distributions to sample from
param_dist = {
            'n_estimator': [100, 200, 400, 600],
            #  'l1_ratio': stats.uniform(0, 1),
            #  'alpha': loguniform(1e-4, 1e0)
              }

# run randomized search
n_iter_search = 20
random_search = RandomizedSearchCV(cls_model, param_distributions=param_grid,
                                   n_iter=n_iter_search)

start = time()
random_search.fit(X_train, y_train)
print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))
report(random_search.cv_results_)



RandomizedSearchCV took 1220.71 seconds for 20 candidates parameter settings.
Model with rank: 1
Mean validation score: 0.914 (std: 0.004)
Parameters: {'classifier__n_estimators': 190, 'classifier__criterion': 'gini'}

Model with rank: 2
Mean validation score: 0.914 (std: 0.005)
Parameters: {'classifier__n_estimators': 170, 'classifier__criterion': 'entropy'}

Model with rank: 3
Mean validation score: 0.914 (std: 0.003)
Parameters: {'classifier__n_estimators': 150, 'classifier__criterion': 'entropy'}



In [175]:
#cls_model=CV.best_estimator_
cls_model=rf
from sklearn.model_selection import cross_val_score
# Multiply by -1 since sklearn calculates *negative* MAE
scores = -1 * cross_val_score(cls_model, X_valid, y_valid,
                              cv=5,
                              #scoring='neg_mean_absolute_error'
                              )

print("MAE scores:\n", scores)

MAE scores:
 [-0.89141165 -0.87857848 -0.85982231 -0.88647581 -0.89239882]


In [180]:
X_test_y = cls_model.predict_proba(X_test)
df = pd.concat([pd.Series(X_test.index),  pd.Series(X_test_y[:,1])], axis=1)
df.shape
df.to_csv('output/p7.csv', index=False, header=False)
#df_train.to_csv('output/p7_train.csv', index=False)
#df_test.to_csv('output/p7_test.csv', index=False)

(2551, 2)

In [186]:
X_test_y = cls_model.predict_proba(X_test)
df = pd.concat([pd.Series(X_test.index),  pd.Series(X_test_y[:,1])], axis=1)
df = df.groupby('Opportunity_ID').mean()
df
df.to_csv('output/p7b.csv', index=True, header=False)
#df_train.to_csv('output/p7_train.csv', index=False)
#df_test.to_csv('output/p7_test.csv', index=False)